In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

from tqdm import tqdm
from collections import defaultdict, Counter

from dask.distributed import Client, progress
import dask.bag as db

import nyata2017

# Big Data testközelből
<i>Fülöp András - Data Scientist @ Lensa|Gitential</i>

<b>NYATA2017</b>, 2017 július 22.

# Python

> *A programozás a legközelebbi dolog a szupererőhöz.*  
> Drew Houston (Dropbox)

In [ ]:
print "Hello NYATA!"

# Hogyan dolgozunk fel adatokat?

## Magyar Elektronikus Könyvtár letölthető könyvei

In [ ]:
bookpaths = nyata2017.list_books('./docs/')
len(bookpaths)

In [ ]:
with open(bookpaths[0]) as bookfile:
    print bookfile.readline()[:140]

In [ ]:
books = []
for bookpath in tqdm(bookpaths[:1000]):
    with open(bookpath) as bookfile:
        books.append(bookfile.readline())

In [ ]:
np.mean([len(book) for book in books])

In [ ]:
fig, ax = plt.subplots()
_ = ax.hist([len(book) for book in books], bins=1000)

## Melyek a MEK leggyakoribb szavai?

In [ ]:
wordcount = defaultdict(int)

for book in tqdm(books):
    for word in book.split():
        wordcount[word.lower()] += 1

In [ ]:
for word, cnt in Counter(wordcount).most_common(10):
    print word, cnt

# Hogyan dolgozunk fel nagy adatokat?

## Párhuzamosítsunk!

In [ ]:
from dask.distributed import Client, progress
import dask.bag as db

In [ ]:
client = Client()
books = db.read_text('./docs/00*.txt')
wordcount = (books.str.lower()
                  .str.split()
                  .flatten()
                  .frequencies()
                  .topk(10, lambda x: x[1])) # TODO: egy fv-t map-elni

In [ ]:
future = client.compute(wordcount)
progress(future)

In [ ]:
for word, cnt in client.gather(future):
    print word, cnt
    
client.close()

# Hogyan dolgozzunk fel Big Data-t?
## Map-Reduce paradigma

In [ ]:
# TODO: wordcount map-reduce implementáció

TODO: Képek hadoop-ról, spark-ról, stb.

TODO: Kép Nvidiáról, GPU-ról, stb.

# Milyen lehetőségek nyílnak meg a Big Data által?

# Kitérő: Neurális hálózatok

## Perceptron model
<img src="pics/neuron.png" width=400 align="left"/>

## Többrétegű hálózat
<img src="pics/mlp.png" width=400 align="left" source="https://github.com/nikolaypavlov/MLPNeuralNet">

# Mi az a mélytanulás?

<img src="pics/mgc.gif"/>

<img src="pics/deepdream.jpg" source="https://artofericwayne.com/2015/07/08/google-deep-dream-getting-too-good/"/>

<img src="pics/deeprebrandt.jpg" source="http://www.wired.co.uk/article/new-rembrandt-painting-computer-3d-printed"/>

<img src="pics/deeptransform.jpg" source="https://deepart.io/"/>

> *PANDARUS:*  
> *Alas, I think he shall be come approached and the day*  
> *When little srain would be attain'd into being never fed,*  
> *And who is but a chain and subjects of his death,*  
> *I should not sleep.*  
> ...  
>   
> *Clown:*  
> *Come, sir, I will make did behold your worship.*  
>   
> *VIOLA:*  
> *I'll drink it.*  

\- William RNNspear

<img src="pics/deeppaper.jpg" source="http://karpathy.github.io/2015/05/21/rnn-effectiveness/">

Szóvektorokról ide valami grafikont

# Mik azok az ajánló rendszerek?

<img src="pics/logos.png">

# Collaborative filtering

<img src="pics/cfinteract.png" />

<img src="pics/cfmatrix.png" />

# Tartalom alapú módszerek 

<img src="pics/cbinteract.png" />

# A nagy testvér figyel?

# Köszönöm megtisztelő figyelmüket!

<img src="pics/twitter.png" align='left' style='margin-right: 5px; margin-left: 15px' /><b>twitter.com/fulibacsi</b><br>

<img src="pics/github.png" align='left' style='margin-right: 5px; margin-left: 15px' /><b>github.com/fulibacsi</b>

**Források:**

- Dokumentumok:
    - mek.oszk.hu


- Deep learning képek:
    - deep dream: https://artofericwayne.com/2015/07/08/google-deep-dream-getting-too-good/
    - Rembrandt 3d nyomtatás: http://www.wired.co.uk/article/new-rembrandt-painting-computer-3d-printed
    - Képek átalakítása festői stílusokban: https://deepart.io/
    - Generatív nyelvek: http://karpathy.github.io/2015/05/21/rnn-effectiveness/